In [15]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 1

In [ ]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0])  # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 1


100%|██████████| 320/320 [00:01<00:00, 165.45it/s]


(0.626430863757238,
 0.3835124999999999,
 0.476475,
 0.3409563607062833,
 0.37728303309835154)

In [17]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

100%|██████████| 200/200 [00:00<00:00, 299.93it/s]


{'average': 0.57505,
 'direct_effect': 0.31637500000000013,
 'spillover_effect': -0.02108749999999987,
 'psi_1_gamma': 0.6727875000000002,
 'psi_0_gamma': 0.3564125000000001,
 'psi_zero': 0.37749999999999995}

In [18]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr_em at 0x1275588b0> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.58394 ± 0.05275
direct_effect: 0.33663 ± 0.10541
spillover_effect: -0.27182 ± 0.11030
psi_1_gamma: 0.68493 ± 0.07552
psi_0_gamma: 0.34830 ± 0.06119
psi_zero: 0.62012 ± 0.14402


In [19]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr at 0x106df8dc0> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.58659 ± 0.04226
direct_effect: 0.32972 ± 0.07469
spillover_effect: -0.02078 ± 0.04650
psi_1_gamma: 0.68536 ± 0.06048
psi_0_gamma: 0.35564 ± 0.03931
psi_zero: 0.37642 ± 0.03819
